# Import libraries

In [1]:
import numpy as np
import keras
from keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow_datasets'